<h2>Scrape and Imports</h2>

In [221]:
# All the imports

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

import requests
from pandas.io.json import json_normalize

import numpy as np
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!pip install folium==0.5.0
import folium 

Montreal raw Data

In [222]:
{
    "tags": [
        "remove-input",
    ]
}

import types
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_a0176b9ee52d41f7bf60b67d1fc261ec = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='QtnPVDXw4Sy4rO00HgJOCQIoMtS09QYV_cbymcVY43eG',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

In [223]:

body = client_a0176b9ee52d41f7bf60b67d1fc261ec.get_object(Bucket='capstone-donotdelete-pr-tn7aynsygnvldo',Key='quartierreferencehabitation.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_3 = pd.read_csv(body)
df_data_3.head()


,no_qr,Borough,no_arr,Neighbourhood,nom_mun
0,53,Rivière-des-Prairies,33.0,Rivière-des-Prairies–Pointe-aux-Trembles,Montréal
1,24,Beaurivage,22.0,Mercier–Hochelaga-Maisonneuve,Montréal
2,23,Tétreaultville,22.0,Mercier–Hochelaga-Maisonneuve,Montréal
3,22,Dupéré,22.0,Mercier–Hochelaga-Maisonneuve,Montréal
4,25,Guybourg,22.0,Mercier–Hochelaga-Maisonneuve,Montréal


In [224]:
mount = df_data_3.drop(columns=['no_qr', 'nom_mun','no_arr']).rename(columns={"Neighbourhood":"Borough","Borough":"Neighbourhood"})
mount.shape

(91, 2)

In [225]:

body = client_a0176b9ee52d41f7bf60b67d1fc261ec.get_object(Bucket='capstone-donotdelete-pr-tn7aynsygnvldo',Key='Neighbourhoods.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_5 = pd.read_csv(body)
df_data_5.head()


,Neighbourhood,Latitude,Longitude
0,Rivière-des-Prairies,45.6677,-73.5462
1,Beaurivage,45.5982,-73.5139
2,Tétreaultville,45.6018,-73.5246
3,Dupéré,45.6046,-73.5357
4,Guybourg,45.5742,-73.5206


Time to merge!

In [226]:
# Combine the two data sets
city = mount.merge(df_data_5, how='inner', on='Neighbourhood')
city.head()

,Neighbourhood,Borough,Latitude,Longitude
0,Rivière-des-Prairies,Rivière-des-Prairies–Pointe-aux-Trembles,45.6677,-73.5462
1,Beaurivage,Mercier–Hochelaga-Maisonneuve,45.5982,-73.5139
2,Tétreaultville,Mercier–Hochelaga-Maisonneuve,45.6018,-73.5246
3,Dupéré,Mercier–Hochelaga-Maisonneuve,45.6046,-73.5357
4,Guybourg,Mercier–Hochelaga-Maisonneuve,45.5742,-73.5206


In [227]:
print('The dataframe has {} boroughs and {} neighbourhoods.'.format(
        len(city['Borough'].unique()),
        city.shape[0]
    )
)

The dataframe has 20 boroughs and 91 neighbourhoods.


In [228]:
address = 'Montreal, QC'

geolocator = Nominatim(user_agent="qb_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Montreal are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Montreal are 45.4972159, -73.6103642.


In [229]:
# create map of Quebec City using latitude and longitude values
map_quebec = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(city['Latitude'], city['Longitude'], city['Borough'], city['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_quebec)  
    
map_quebec

<h2>Finished with the Data. Setting up the maps!</h2>

In [230]:
# Foursqaure ID

import json
{
    "tags": [
        "remove-input",
    ]
}
CLIENT_ID = '4F2TWL40HEUEKJXU01ZUMRE4ZNYOGGTRNMPLGRVNJT33J1KJ' # your Foursquare ID
CLIENT_SECRET = 'O5MCRNOS2HOODSSYWX15PYEVB05WZMPY2VGB3GC1K4KQBCFG' # your Foursquare Secret
ACCESS_TOKEN = 'KFUMX2PWRS3FVCUZRGFCYPKLJ1VEKWDLC2NWLHOW24M3S5C4'
v = '20200605' # Foursquare API version
limit = 100 # A default Foursquare API limit value
radius = 1000 

First Neighbourhood

In [231]:
city.loc[5, 'Neighbourhood']

'Longue-Pointe'

In [232]:
neighbourhood_latitude = city.loc[5, 'Latitude']
neighbourhood_longitude = city.loc[5, 'Longitude'] 

neighbourhood_name = city.loc[5, 'Neighbourhood'] 

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Longue-Pointe are 45.5781, -73.5382.


Create Foursquare link

In [233]:
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    v, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    limit
)

In [242]:
results = requests.get(url).json()
results

{'meta': {'code': 429,
  'errorType': 'quota_exceeded',
  'errorDetail': 'Quota exceeded',
  'requestId': '5ffb72e72e715e58deb64a59'},
 'response': {}}

In [339]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    if cateoryId.equals("4d4b7105d754a06374d81259"):
        return categories_list[0]['name']
    else:
        return None
    
for categoryId in row['categories']:
    if (categoryId.equals("4d4b7105d754a06374d81259")):
        return categories_list[0]['name']
    else:
        return None

SyntaxError: 'return' outside function (<ipython-input-339-d1fce35814ec>, line 19)

In [266]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

KeyError: 'groups'

In [267]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

35 venues were returned by Foursquare.


<h3>Now that that works, Here's the repetition for the rest of them:</h3>

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            v, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
        if categoryId ['4d4b7104d754a06370d81259','4d4b7105d754a06377d81259','4d4b7105d754a06378d81259']:
        return None
    else:
        return categories_list[0]['name']
    
    return(nearby_venues)

In [ ]:
# creating a list of categories to remove from our dataframe because they are not restaurants

removal_list = ['Arts & Entertainment','College & University','Event','Outdoors & Recreation','Nightlife Spot','Professional & Other Places','Residence','Shop & Service','Travel & Transport']

nearby_venues2 = nearby_venues.copy()


#getting a clear dataframe of just restaurants
nearby_venues2 = nearby_venues2[~nearby_venues2['categories'].isin(removal_list)]
nearby_venues2.head()


In [ ]:
city_venues = getNearbyVenues(names=city['Neighbourhood'],
                                   latitudes=city['Latitude'],
                                   longitudes=city['Longitude']
                                  )

In [ ]:
city_venues.groupby('Neighbourhood').count()

In [ ]:
print('There are {} uniques categories.'.format(len(city_venues['Venue Category'].unique())))

In [ ]:
city_venues.head()

<h2> Now let's look at what's in Montreal</h2>

In [ ]:
# one hot encoding
city_onehot = pd.get_dummies(city_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
city_onehot['Neighbourhood'] = city_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [city_onehot.columns[-1]] + list(city_onehot.columns[:-1])
city_onehot = city_onehot[fixed_columns]

city_onehot.head()

In [ ]:
city_grouped = city_onehot.groupby('Neighbourhood').mean().reset_index()
city_grouped.head()

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = city_grouped['Neighbourhood']

for ind in np.arange(city_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(city_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, categoryID = '4fa862b3e4b0ebff2f749f06', radius=1000, limit=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        categoryId = '4fa862b3e4b0ebff2f749f06' # ID of 'Food'
        url = 'https://api.foursquare.com/v2/venues/search?&categoryId={}&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            v, 
            lat, 
            lng, 
            categoryId, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']['categories']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
city_venues = getNearbyVenues(names=city['Neighbourhood'],
                                   latitudes=city['Latitude'],
                                   longitudes=city['Longitude']
                                  )

city_venues.groupby('Neighbourhood').count()